In [ ]:
%pip install mpqp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of awscli to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of awscli to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter

In [10]:
from mpqp import QCircuit
from mpqp.gates import H

def prepare_initial_state(n_precision_qubits, eigenstate_circuit):
    # Create an empty circuit with total number of qubits
    n_eigenstate_qubits = eigenstate_circuit.nb_qubits
    total_qubits = n_precision_qubits + n_eigenstate_qubits
    circ = QCircuit(nb_qubits=total_qubits, label="QPE Circuit")

    # Define the precision qubits and the circuits qubits range indices
    precisions = range(n_precision_qubits)
    circuits = range(n_precision_qubits, total_qubits)

    # Apply Hadamard gates to precision qubits (index 0 to n-1)
    for i in range(n_precision_qubits):
        circ.add(H(i))

    # Append the given eigenstate circuit offset to the "bottom" qubits
    circ.append(eigenstate_circuit, qubits_offset=n_precision_qubits)

    # Return circuit in state |+>^n \otimes |eigenstate>
    return circ, precisions, circuits

In [11]:
from mpqp import QCircuit
from mpqp.gates import CNOT, TOF, ControlledGate

def controlled_unitary(circuit, unitary, control_qubit, offset):
    # cu_circuit = QCircuit(nb_qubits=10) # TODO fix
    cu_circuit = QCircuit(nb_qubits=circuit.nb_qubits, label="Controlled Unitary Circuit")

    for gate in unitary.gates:
        print(f"Processing gate {gate}")
        if len(gate.targets) == 1: # Single target gate
            target = gate.targets[0] + target_offset
            if isinstance(gate, CNOT): # CNOT -> TOF
                control = gate.controls[0] + target_offset
                cu_circuit.add(TOF([control_qubit, control], target))
            else: # Other single target gates becomes controlled versions
                cu_circuit.add(ControlledGate([control_qubit], [target], gate))
    return cu_circuit

In [13]:
def apply_controlled_unitary_powers(initial_circuit, precisions, eigenstate_qubits, unitary):
    # Loop through precision qubits in reverse order
    for i in precisions[::-1]:
        # Apply controlled unitary operator 2^(n-i-1) times (1, 2, 4, ...)
        for _ in range(2 ** (len(precisions) - i - 1)):
            controlled_unitary(initial_circuit, i, len(precisions), unitary)


In [14]:
from mpqp.gates import *
from mpqp.execution.result import Result
from mpqp import QCircuit, Barrier
from mpqp.execution import run, IBMDevice
from math import floor
import numpy as np


class InverseQFT(QCircuit):

    def __init__(self,n_qubits):
        super().__init__(n_qubits, nb_cbits=n_qubits)
        self._build()

    def _build(self):
        self.add([SWAP(i, self.nb_qubits - 1 - i)
            for i in range(int(floor(self.nb_qubits / 2)))])
        j = self.nb_qubits - 1
        while j >= 0:
            self.add(Barrier())
            self.add([CRk(i+1 - j, i, j).inverse() for i in range(j+1, self.nb_qubits)])
            self.add(H(j))
            j -= 1

In [30]:
from mpqp.measures import BasisMeasure
from numpy import pi as PI

def QPE(unitary, eigenstate_circuit, n_precision_qubits):
    # n = number of precision qubits
    # m = number of eigenstate qubits
    # unitary = unitary circuit to apply
    # eigenstate_circuit = circuit that prepares the initial eigenstate

    #### 1. Prepare the initial state |+>^n \otimes |eigenstate> ####
    # Create an empty circuit with total number of qubits
    n, m = n_precision_qubits, eigenstate_circuit.nb_qubits
    total_qubits = n + m
    circ = QCircuit(nb_qubits=total_qubits, label="QPE Circuit")
    precisions = range(n)

    # Apply Hadamard gates to precision qubits (index 0 to n-1)
    for i in range(n_precision_qubits):
        circ.add(H(i))

    # Append the given eigenstate circuit offset to the "bottom" qubits
    circ.append(eigenstate_circuit, qubits_offset=n_precision_qubits)
    circ.add(Barrier())
    print("Initial state prepared: |+>^n otimes |eigenstate>")

    #### 2. Apply controlled unitary operators ####
    # Loop through precision qubits in reverse order
    print("Applying controlled unitary operations.")
    precisions = range(n)
    for i in precisions[::-1]:
        print(f"Applying controlled unitary for precision qubit {i}")
        # Apply controlled unitary operator 2^(n-i-1) times (1, 2, 4, ...)
        iterations = 2 ** (n - i - 1)
        print(f"Applying controlled unitary {iterations} times.")
        for k in range(iterations):
            print(f"\t{iterations-k} remaining")

            # Transform the unitary circuit to its controlled version with control qubit at position i
            for gate in unitary.gates:
                if len(gate.targets) == 1:
                    target = gate.targets[0] + n
                    if isinstance(gate, CNOT): # CNOT -> TOF
                        control = gate.controls[0] + n
                        circ.add(TOF([i, control], target))
                    elif isinstance(gate, X):
                        circ.add(CNOT(i, target))
                    elif isinstance(gate, S):
                        circ.add(CP(PI/2, i, target))
                    elif isinstance(gate, Y):
                        circ.add(S(target))
                        circ.add(CNOT(i, target))
                        circ.add(S_dagger(target))
                    elif isinstance(gate, Z):
                        circ.add(CZ(i, target))
                    elif isinstance(gate, T):
                        circ.add(CP(PI/4, i, target))
                    elif isinstance(gate, H):
                        circ.add(Rz(PI, target))
                        circ.add(Rz(PI/2, target))
                        circ.add(CNOT(i, target))
                        circ.add(Rz(-PI, target))
                        circ.add(Rz(-PI/2, target))
                    else:
                        raise ValueError("Unknown single-target gate in controlled unitary.")
                else:
                    raise ValueError("Only single-target gates are supported in controlled unitary.")
        circ.add(Barrier())
    print("Controlled unitary operations applied.")

    #### 3. Apply inverse QFT to precision qubits ####
    print("Applying inverse QFT to precision qubits.")
    circ.append(InverseQFT(n), qubits_offset=0)
    circ.add(Barrier())
    print("Inverse QFT applied.")

    #### 4. Measure precision qubits ####
    circ.add(BasisMeasure(targets=list(range(n)), shots=1024))
    print("Measurement added to precision qubits.")

    # Add a barrier for clarity
    circ.add(Barrier())
    # circ.pretty_print()
    return circ

In [43]:
from mpqp import *
from mpqp.gates import *
from collections import defaultdict


def extract_phase(unitary, eigenstate_circuit, n_precision_qubits):
    qpe_circuit = QPE(unitary, eigenstate_circuit, n_precision_qubits)

    # Running circuit
    result = run(qpe_circuit, IBMDevice.AER_SIMULATOR_STATEVECTOR)

    mask            = (1 << n_precision_qubits) - 1
    weight_by_k     = defaultdict(float)

    for sample in result.samples:
        k = sample.index & mask
        weight = getattr(sample, "probability", None)
        if weight is None:
            weight = sample.count
        weight_by_k[k] += weight

    # Picking the k with the largest total weight
    best_k = max(weight_by_k, key=weight_by_k.get)

    # Converting to phase ϕ̂ = k / 2ⁿ
    phase_estimate = best_k / (1 << n_precision_qubits)

    print(f"Estimated phase: {phase_estimate}")
    return phase_estimate

In [40]:
def eigenstate_1():
    c = QCircuit(1)
    c.add(H(0))
    return c

def unitary_Z():
    c = QCircuit(1)
    c.add(H(0))
    return c

unitary = unitary_Z()
eigenstate = eigenstate_1()

# Expecting 0 or 0.5
extract_phase(unitary, eigenstate, n_precision_qubits=3)

Initial state prepared: |+>^n otimes |eigenstate>
Applying controlled unitary operations.
Applying controlled unitary for precision qubit 2
Applying controlled unitary 1 times.
	1 remaining
Applying controlled unitary for precision qubit 1
Applying controlled unitary 2 times.
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 0
Applying controlled unitary 4 times.
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Controlled unitary operations applied.
Applying inverse QFT to precision qubits.
Inverse QFT applied.
Measurement added to precision qubits.
Estimated phase: 0.5


0.5

In [41]:
unitary = QCircuit(1)
unitary.add(S(0))
eigenstate = QCircuit(1)
eigenstate.add(X(0))  # |1⟩
n_precision_qubits = 4

# Expecting 0.25
extract_phase(unitary, eigenstate, n_precision_qubits)

Initial state prepared: |+>^n otimes |eigenstate>
Applying controlled unitary operations.
Applying controlled unitary for precision qubit 3
Applying controlled unitary 1 times.
	1 remaining
Applying controlled unitary for precision qubit 2
Applying controlled unitary 2 times.
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 1
Applying controlled unitary 4 times.
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 0
Applying controlled unitary 8 times.
	8 remaining
	7 remaining
	6 remaining
	5 remaining
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Controlled unitary operations applied.
Applying inverse QFT to precision qubits.
Inverse QFT applied.
Measurement added to precision qubits.
Estimated phase: 0.25


0.25

In [42]:
unitary = QCircuit(1); unitary.add(T(0))
eigenstate = QCircuit(1); eigenstate.add(X(0))  # |1⟩
n_precision_qubits = 5

# Expecting 0.125
extract_phase(unitary, eigenstate, n_precision_qubits)

Initial state prepared: |+>^n otimes |eigenstate>
Applying controlled unitary operations.
Applying controlled unitary for precision qubit 4
Applying controlled unitary 1 times.
	1 remaining
Applying controlled unitary for precision qubit 3
Applying controlled unitary 2 times.
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 2
Applying controlled unitary 4 times.
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 1
Applying controlled unitary 8 times.
	8 remaining
	7 remaining
	6 remaining
	5 remaining
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 0
Applying controlled unitary 16 times.
	16 remaining
	15 remaining
	14 remaining
	13 remaining
	12 remaining
	11 remaining
	10 remaining
	9 remaining
	8 remaining
	7 remaining
	6 remaining
	5 remaining
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Controlled unitary operations applied.
Applying inverse QFT to p

0.125